In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
# Parameters
zonas = ['norte', 'sul', 'leste', 'oeste']
URL='https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']

In [3]:
# Functions
def scrap(url):
    c=requests.get(url).content
    soup=BeautifulSoup(c)
    aluguel_tags = soup.find_all('span', {'class':'price-tag-fraction'})
    alugueis_lista = [p.text for p in aluguel_tags]
    enderecos_tags = soup.find_all('span', {'class':'ui-search-item__group__element'})
    enderecos_lista=[end.text for end in enderecos_tags]
    enderecos_lista=enderecos_lista[1::2]
    area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})
    area_quartos_lista=[aq.text for aq in area_quartos_tags]
    df=convert_df(zona, area_quartos_lista, enderecos_lista, alugueis_lista)
    return df

def convert_df(zona, area_quartos_lista, enderecos_lista, alugueis_lista):
    zonas_lista = [zona]*len(alugueis_lista)
    data = {'zona': zonas_lista,
            'area_quartos': area_quartos_lista,
            'enderecos_lista':enderecos_lista,
            'aluguel': alugueis_lista}
    df = pd.DataFrame(data)
    return df

In [4]:
# Execution
df_lista = []
for zona in zonas:
  for pg in paginas:
    url=URL.format(zona, pg)
    print('Coletando URL:', url)
    try:
      df=scrap(url)
    except Exception as e:
      print('Erro ao coletar URL:', url)
      print(e)
    df_lista.append(df)
    sleep(1)
df_final = pd.concat(df_lista)

Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_289
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Coletando URL: https://imoveis.mercadolivre.com.br/casas/aluguel

In [5]:
df_final.head()


,zona,area_quartos,enderecos_lista,aluguel
0,norte,70 m² construídos1 quarto,"Nobrega De Souza Coutinho 05, Jardim Peri, São...",600
1,norte,80 m² construídos2 quartos,"Rua José Eduardo Raduan 5, Parque Edu Chaves, ...",1.299
2,norte,50 m² construídos1 quarto,"Rua Campo Belo Do Sul 113, Parque Vitória, São...",1.000
3,norte,56 m² construídos2 quartos,"Narita, Jardim Japão, São Paulo Zona Norte, Sã...",1.300
4,norte,140 m² construídos3 quartos,"João Miranda De Moura 22, Tremembé, São Paulo ...",1.950


In [9]:
df_final.to_csv('alugueis.csv') 